In [2]:
# работа с таблицами
import numpy as np
import pandas as pd

# работа с графиками
import matplotlib.pyplot as plt
import seaborn as sns

# вспомогательные методы
from IPython.display import display
from transliterate import translit
import warnings

### Функции

In [3]:
# анализ категорий, неявных дубликатов и ошибок (явных аномалий) в данных
def quick_look(df:pd.DataFrame,name:str="") -> None:

    result = pd.concat((
        df.dtypes.to_frame(name="dtype").T,
        df.nunique().to_frame(name="nunique").T,
        round((df.apply(pd.Series.duplicated).sum() / df.count()).to_frame(name="duplicates").T,2),
        round(df.isna().mean().to_frame(name="NA").T,2),
        pd.DataFrame([["─"] * df.shape[1] ],columns=df.columns,index=["─ ex:"]),
        df.head(1),
        df.iloc[1:-2].sample(1),
        df.tail(1),
        pd.DataFrame([["─"] * df.shape[1]],columns=df.columns,index=["─ m:"]),
        df.min().to_frame(name="min").T,
        df.loc[:,df.dtypes != object].mean().to_frame(name="mean").T,
        df.max().to_frame(name="max").T,
    ))

    result.index.name = name

    return result

### Загрузка датасета

In [4]:
df_clients = pd.read_excel('Выгрузка для ЯП_2.xlsx', sheet_name='Доноры')
df_payments = pd.read_excel('Выгрузка для ЯП_2.xlsx', sheet_name='Пожертвования')

In [5]:
df_clients = df_clients.rename(columns={"Дата последней email активности":"email_action",'Город':'city','Пол':'gender_male'})

df_clients_gend = sorted(df_clients.gender_male.dropna().unique())
df_clients.gender_male = df_clients.gender_male.replace({val:i for i, val in enumerate(df_clients_gend)})

In [6]:
df_payments = df_payments\
    .rename(columns=
            {"Donor ID":"ID",
            'Дата платежа':'pay_date',
            'Рекуррентный':'recurrentis',
            'Сумма платежа':'pay_sum',
            'Назначение платежа':'pay_purpose',
            'Канал оплаты':'pay_method'})
df_payments_rec = sorted(df_payments.recurrentis.unique(),reverse=True)
df_payments.recurrentis = df_payments.recurrentis.replace({val:i for i, val in enumerate(df_payments_rec)})

df_payments.pay_method = df_payments.pay_method.str.strip().str.lower()

In [7]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    display(quick_look(df_clients))

,ID,gender_male,city,email_action
,,,,
dtype,object,float64,object,datetime64[ns]
nunique,3683,2,234,653
duplicates,0.0,1.51,3.4,1.65
NA,0.0,0.34,0.72,0.5
─ ex:,─,─,─,─
0,AF575B85-917F-475D-8875-00055586E05B,NaN,NaN,NaN
3553,11C0C9F9-2A6D-4E28-A89C-F8109D4D930C,0.0,Москва,2022-07-18 16:29:48
3682,B7B85C92-A071-4D15-A8AD-FFF3107199DA,0.0,Волгоград,2020-10-08 17:42:07
─ m:,─,─,─,─


In [8]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    display(quick_look(df_payments))

,ID,pay_date,pay_sum,recurrentis,pay_purpose,pay_method
,,,,,,
dtype,object,datetime64[ns],float64,int64,object,object
nunique,3683,13386,373,2,893,4
duplicates,0.74,0.05,0.97,1.0,1.03,1.0
NA,0.0,0.0,0.0,0.0,0.09,0.0
─ ex:,─,─,─,─,─,─
0,DE5393F9-9585-4057-B7FB-24B69697FC08,2022-07-03 19:49:48,300.0,1,ПОЖЕРТВОВАНИЕ undefined,Банковские карты
12665,4BA0AD99-4220-4C89-8476-3F9B51372DCD,2021-11-23 20:00:47,1000.0,1,ПОЖЕРТВОВАНИЕ,Банковские карты
14042,EAD75CE6-0F33-478E-A37E-712C66DB9082,2020-05-27 22:49:33,10000.0,1,ПОЖЕРТВОВАНИЕ,Банковские карты
─ m:,─,─,─,─,─,─


In [9]:
#смотрим, совпадают ли ID
set(df_payments.ID) - set(df_clients.ID), set(df_clients.ID) - set(df_payments.ID)

(set(), set())

In [89]:
purp_replaces = {
    "фонд":['пожертвование',
        'пожертвование в фонд помощи детям-отказникам и детям-сиротам "бюро добрых дел"',
        'пожертвование  undefined',
        'добровольное пожертвование. ндс не облагается',
        'пожертвование без назначения',
        'на работу фонда',
        'добровольное пожертвование',
        'добровольный взнос',
        'благотворительное пожертвование. ндс не облагается',
        'благотворительность',
        'помощь',
        'помощь',
        'благотворительное пожертвование',
        'благотворительность ',
        'благотворительный взнос',
        'пожертвование  всем',
        'пожертвование  ',
        'пожертвование',
        'благотворительный взнос',
        'пожертвование',
        'детям',
        'добровольное пожертвование',
        '6162;61 62;сумма 100 руб.;комиссия 0 руб;дата оплаты 09/05/2019;',],
    "фонд.смс":['sms bdd'],
    "фонд.клуб_друзей":['пожертвование клуб друзей'],
    "фонд.ресурсный_центр":['на ресурсный центр'],
    "фонд.окно_в_мир":['на программу окно в мир']
}

purp_conditions = {
    "целевой.обучение":["на обучение","На обучение","Московские каникулы","На тьютора","обучение "],
    "целевой.псих_помощь":["На психологическую помощь"], 
    "целевой.проживание":["На проживание",],
    "фонд.соц_квартиры":["Соц.квартиры","Социальные квартиры","соц.квартиры","соц.квартира",'социальные гостиницы'],
    "фонд.история_успеха":["История успеха"],
    "фонд.школа_семьи":["Школа семьи"],
    "фонд.окно_в_мир":['Окно в мир'],
    "фонд.трудоустройство":['На трудоустройство','успешное будущее','путевка в будущее'],
    "фонд.центр_профориентации":["Центр профориентации"],
    "фонд.дети_вместо_цветов":['Дети вместо цветов'],
    "фонд.подари_профессию":['Подари профессию','будующая профессия','будущая профессия','найти себя'],
    "фонд.ресурсный_центр":['Ресурсного центра','ресурс'],
    "фонд.школа_нетрудных_подростков":['нетрудных подростков','школа подростков'],
    "фонд.живые_мечты":['живые мечты'],
    "фонд.детский_дом":["детскому дому","детских домов","детский дом",'детского дома'],
    "возврат":[ 'возврат',"ВОЗВРАТ НЕИСПОЛЬЗОВАННЫХ СРЕДСТВ","Возврат неиспользованных денежных средств",
                'Возврат ошибочно переведенных средств', 'Возврат средств'],
    "целевой":[ "День рождения со смыслом","Артём","Артем","Ксении","Февронии","Вере","Андрея","Саша","Лиза","Алина","Юре",
                "Софья", "Глеба", "Саше", "Катя", "для ","Амиру","стасу",],  
    "фонд":['Пожертвование в БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ПОМОЩИ ДЕТЯМ-ОТКАЗНИКАМ И ДЕТЯМ-СИРОТАМ',"отказникам","на ",
            'БЛАГОТВОРИТЕЛЬНОЕ ПОЖЕРТВОВАНИЕ','оплата по договору',"пополнение","добровольное пожертвование",
            'благотворительный взнос;','перевод с карты','разово','пожертвование','благотворительный взнос'],
}
temp_series = df_payments.pay_purpose.str.lower().str.strip()

for target, values in purp_conditions.items():
    for value in values:
        temp_series.loc[temp_series.str.contains(value.lower()).fillna(False)] = target

for target, values in purp_replaces.items():
    temp_series = temp_series.replace(values,target)

print(temp_series.value_counts(dropna=False))
df_payments['pay_purpose_category'] = temp_series

df_payments.loc[df_payments.pay_method == "sms",'pay_purpose_category'] = 'фонд.смс'
df_payments.loc[df_payments.pay_purpose.isna(),'pay_purpose_category'] = 'фонд'
df_payments.loc[df_payments.pay_purpose == 500,'pay_purpose_category'] = 'фонд'

df_payments['pay_purpose_cat'] = None

for cat in ['фонд','целевой','возврат']:
    mask = df_payments['pay_purpose_category'].str.startswith(cat)
    df_payments.loc[mask.fillna(False),'pay_purpose_cat'] = cat

print("-"*50,"\n",df_payments.pay_purpose_cat.value_counts(dropna=False))

print("-"*50,"\n",f'Осталось необработано: {df_payments.pay_purpose_cat.isna().sum():.0f} строк')

фонд                               10897
NaN                                 1270
целевой                             1133
фонд.смс                             253
целевой.проживание                   114
целевой.обучение                      81
целевой.псих_помощь                   51
фонд.окно_в_мир                       45
фонд.история_успеха                   41
фонд.ресурсный_центр                  37
фонд.школа_семьи                      36
возврат                               26
фонд.трудоустройство                  20
фонд.подари_профессию                  8
фонд.соц_квартиры                      8
фонд.дети_вместо_цветов                8
фонд.школа_нетрудных_подростков        6
фонд.детский_дом                       4
фонд.живые_мечты                       4
фонд.центр_профориентации              1
Name: pay_purpose, dtype: int64
-------------------------------------------------- 
 фонд       12638
целевой     1379
возврат       26
Name: pay_purpose_cat, dtype: int64
--------